# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/camilo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [18]:
import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

[nltk_data] Downloading package punkt to /Users/camilo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/camilo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(max_df = 0.5, ngram_range = (1, 1), tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10), n_jobs=-1))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train,Y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(max_df=0.5,
                                                                                  tokenizer=<function tokenize at 0x7fe66bea5dc0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10),
                                       n_jobs=-1))])

In [21]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [23]:
Y_test.shape

(6554, 35)

In [24]:
Y_pred.shape

(6554, 35)

Let's make sure that that there aren't are some labels in y_true, which dont appear in y_pred making y_pred ill-defined, and be the reason for the problem after running the classification report:

UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)

In [22]:
set(Y_test) - set(Y_pred)

TypeError: unhashable type: 'numpy.ndarray'

In [25]:
index = [str(x) for x in list(Y_test.index)]
category_names = list(Y.columns)
Y_pred = pd.DataFrame(Y_pred, index= index , columns = category_names )

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
print(classification_report(Y_test, Y_pred, target_names= category_names))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [10],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
}
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_macro', cv=None, n_jobs=-1,verbose=10) 
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

The tuned model has parameters n= 10 for the RandomForestClassifier and max_df = 0.5 for the CountVectorizer feature extraction method. Here's the accuracy, precision and recall of the tuned model:

In [ ]:
print(classification_report(Y_test, Y_pred, target_names= category_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We tried a K-neighbors classifier with K=8, but the performance was poorer than the Random Forest Classifier. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=8), n_jobs=-1))
])



X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train,Y_train)

Y_pred = pipeline2.predict(X_test)
index = [str(x) for x in list(Y_test.index)]
category_names = list(Y.columns)
Y_pred = pd.DataFrame(Y_pred, index= index , columns = category_names )

print(classification_report(Y_test, Y_pred, target_names= category_names))

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(cv, open('train_classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.